In [10]:
import pandas as pd
import ipywidgets as widgets
import camelot
import re

# Pulling Indigenous Information from CER Final Reports

Description of app: This app allows users to load a PDF they would like to extract tables from. The app has been specifically designed to focus on tables containing indigenous information in the CER final reports. It can still work for other types of tables, but it isn't guarenteed it will work properly.

## Caveats

The app was created with the consideration that all CER final reports are the same. If the structure changes, it may not extract the tables as well.

## Uploading a PDF of the CER Final Report

Please upload the PDF you are currently interested in. Afterwards, you will be asked to fill in some information and then the table you are looking at. You can only extract one table at a time. Give the page numbers of the table and it will be extracted.

In [2]:
btn_upload = widgets.FileUpload(accept='.pdf', multiple=True)
display(btn_upload)

FileUpload(value={}, accept='.pdf', description='Upload', multiple=True)

In [29]:
def extract_table(btn_upload):
    [uploaded_file] = btn_upload.value
    file_path = uploaded_file

    for name, file_info in btn_upload.value.items():
        with open (name, 'wb') as file:
            file.write(file_info['content'])
            
    name_pattern = '([^//]+)$'
    name_of_document = re.findall(name_pattern, file_path)
    
    pages_input = input("Enter the list of pages separated by space (e.g. 129 130 131): ")
    pages = pages_input.split()

    name_of_section = input()
    name_of_appendix = input()

    tables = camelot.read_pdf(uploaded_file, copy_text=['v'], strip_text = '\n', line_scale=40, f = 'excel', flavour = 'lattice', pages='229, 230')
    
    for i, page in enumerate(pages):
        tables[i].df['page_number'] = page
        tables[i].df['name_of_document'] = name_of_document[0][9:-18]
        # NEB_-_Report_-_Enbridge_-_Line_3_Replacement_Detailed_Assessment
        tables[i].df['date_of_document'] = file_path[-8:-4]
        tables[i].df['date_of_section'] = name_of_section
        tables[i].df['name_of_appendix'] = name_of_appendix
        
    df = tables[0].df.append([tables[x].df for x in range(1, len(tables))]).reset_index(drop=True)
    
    s = df[1].str.split('  ').apply(pd.Series, 1).stack()
    s.index = s.index.droplevel(-1)
    s.name = '1'
    
    del df[1]
    df = df.join(s)
    
    df['1'] = df['1'].str.replace('', '')
    df['1'] = df['1'].str.replace('  ', '')
    
    df.to_excel(name_of_document[0][9:-18] + '_' + pages[0] + '_' + pages[-1] + '.xlsx')
        
    

In [30]:
extract_button = widgets.Button(description="Extract Table")
extract_button.on_click(extract_table(btn_upload))

Enter the list of pages separated by space (e.g. 129 130 131): 129 130
Name of section
Name of appendix
